<a href="https://colab.research.google.com/github/wooookim/ML-DL/blob/main/10_2_%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D%EA%B3%BC_%EA%B7%B8%EB%A6%AC%EB%93%9C%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [14]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42)

from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))


'''
교차검증
- 검증세트를 떼어 내어 평가하는 과정을 여러번 반복
'''
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)  # 기본값 5-fold
print(scores)  # 훈련시간, 검증시간, 검증점수

import numpy as np
print(np.mean(scores['test_score']))

'''
cross_validate
- 훈련세트를 섞어서 폴드를 나누지 않음
 -> 교차 검증 시 훈련세트를 섞으려면 분할기(splitter) 지정 필요
- 기본적으로 회귀 모델의 경우 KFold 분할기를 사용
- 분류 모델의 경우 타겟 클래스를 골고루 나누기 위해 StratifiedKFold 사용
'''
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))


'''
하이퍼파라미터 튜닝
- 하나의 최적 매개변수를 찾고 고정시켜서 다른 매개변수를 변경시키면 더이상 최적이 아니게 됨
- 반복문으로 최적을 찾을 수 있지만 사이킷런에서 제공하는 그리드서치가 있음

그리드서치
- 탐색과 교차검증을 동시에 수행
'''
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)  # min_impurity_decrease 값을 바꿔가면서 5번 실행, 값마다 5폴드 교차 검증 수행 => 5*5 수행
gs.fit(train_input,train_target)  # 교차검증에서 최적 파라미터를 찾으면 전체 훈련세트로 모델을 다시 만들어야 된다
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
print(gs.best_params_)
print(gs.cv_results_['mean_test_score'])   # 각 매개변수에서 수행한 교차검증의 평균 점수

best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

0.9971133028626413
0.864423076923077
{'fit_time': array([0.010674  , 0.01011252, 0.01350379, 0.01362348, 0.00907993]), 'score_time': array([0.00141644, 0.00137687, 0.00148392, 0.00309896, 0.00150442]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487
0.855300214703487
0.8574181117533719
0.9615162593804117
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}


In [17]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), 'max_depth': range(5, 20, 1), 'min_samples_split': range(2, 100, 10)}
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


In [18]:
'''
탐색할 매개변수 간격을 임의로 지정했음

- 매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있음
- 너무 많은 매개변수 조건이 있어 그리드 서치 수행 시간이 오래 걸릴 수 있음

랜덤서치
- 매개변수 값의 목록을 전달하는게 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달
'''
from scipy.stats import uniform, randint   # 각각 실수, 정수를 주어진 샘플에서 균등하게 뽑음
# rgen = randint(0, 10)  0에서 10까지
# rgen.rvs(10)  각각 10번 뽑았을때 나온 횟수

# ugen = uniform(0, 1)   0에서 1사이
# ugen.rvs(10)   비율로 출력

params = {'min_impurity_decrease': uniform(0.0001, 0.001), 'max_depth': randint(20, 50), 'min_samples_split': randint(2, 25), 'min_samples_leaf': randint(1, 25)}

from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params, n_iter = 100, n_jobs = -1, random_state = 42)
gs.fit(train_input, train_target)

print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884
0.86


내일 데이콘에 적용해보기